In [8]:
import pandas as pd
from sqlalchemy import create_engine

In [9]:
Texas_locational_file = "../ETL-Project/CSV's/area_context_texas.csv"
Texas_locational_df = pd.read_csv(Texas_locational_file)
Texas_locational_df.head()

,Housing Units,zipcode,Water Area,Median Home Value,Median Household Income,Population Density,Occupied Housing Units,Population,id,Land Area
0,8102,75001,0.00,258200,57633,3237,7088,12414,750,3.83
1,21373,75002,2.06,186100,91632,1699,20708,63140,750,37.16
2,17747,75006,0.25,146800,56985,2746,16631,46364,750,16.89
3,19296,75007,0.22,169400,78934,4433,18514,51624,750,11.65
4,3074,75009,1.57,190500,87581,92,2812,8785,750,95.42


SyntaxError: unexpected EOF while parsing (<ipython-input-12-0978379c068f>, line 7)

In [14]:
# Create a filtered dataframe from specific columns
Texas_locational_cols=["zipcode", "Median Household Income", "Population",]
Texas_locational_refined= Texas_locational_df[Texas_locational_cols].copy()

# Rename the column headers
Texas_locational_refined = Texas_locational_refined.rename(columns={"zipcode": "Zipcode",
                                                          "Median Household Income": "Median Income","Population": "population"})

# Clean the data by dropping duplicates and setting the index
Texas_locational_refined.drop_duplicates("Zipcode", inplace=True)
Texas_locational_refined.set_index("Zipcode", inplace=True)

Texas_locational_refined.head()

,Median Income,population
Zipcode,,
75001,57633,12414
75002,91632,63140
75006,56985,46364
75007,78934,51624
75009,87581,8785


In [15]:
Texas_locational_refined = Texas_locational_refined.reset_index(drop=False)
Texas_locational_refined


,Zipcode,Median Income,population
0,75001,57633,12414
1,75002,91632,63140
2,75006,56985,46364
3,75007,78934,51624
4,75009,87581,8785
5,75010,75696,21607
6,75013,131878,30347
7,75019,110445,38666
8,75020,40999,21872
9,75021,40000,8036


In [17]:
Texas_locational_refined.to_csv("clean_data.csv", encoding="utf-8", index=False)



In [ ]:
HAR_file="../ETL-Project/CSV's/HAR.csv"
HAR_df= pd.read_csv(HAR_file)
HAR_df.head()

HAR_cols=["Price", "Street", "Zip Code", "City"]
HAR_refined= HAR_df[HAR_cols].copy()
HAR_refined.to_csv("HAR_clean_data.csv", encoding="utf-8", index=False)
HAR_refined

In [ ]:
rds_connection_string = "jon:postgres@localhost:5432/Texas_locational_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
table_names = engine.table_names()
table_names

In [ ]:
# Check if texas_regional_info is in tables
if 'texas_regional_info' in table_names:
   # Get Existing Restaurants
   existing_regional_info = pd.read_sql_query('select * from texas_regional_info', con=engine)
   # Only Add Restaurants Not in Existing
   add_existing_regional_info = Texas_locational_refined.loc[~Texas_locational_refined['Zipcode'].isin(existing_regional_info['Zipcode'])].copy()
   # Add Restaurants
   add_existing_regional_info.to_sql(name='texas_regional_info', con=engine, if_exists='append', index=False)
# If Not in Tables, Add to Tables
else:
   Texas_locational_refined.to_sql(name='texas_regional_info', con=engine, if_exists='append', index=False)

#Texas_locational_refined.to_sql(name='texas_regional_info', con=engine, if_exists='append', index=True)

In [ ]:
pd.read_sql_query(f'select * from {table_names[0]}', con=engine).head()